# Example with C++

This notebooks shows how to **call C++ functions** using the interface from **consav**.

**Platforms:** It only works on **Windows** computers. 

**Compilers:** One of these compilers must be installed

* **vs**: Free *Microsoft Visual Studio 2022 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Free *Intel one API* ([link](https://www.intel.com/content/www/us/en/developer/tools/oneapi/base-toolkit-download.html))

**Computer used for timings:** Windows 10 computer with 2x Intel(R) Xeon(R) Gold 6254 3.10 GHz CPUs (18 cores, 36 logical processes each) and 768 GB of RAM.

# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import numba as nb

# load the BufferStockModel module
from BufferStockModel import BufferStockModelClass

In [2]:
DO_INTEL = True

In [3]:
threads_list = [x for x in np.arange(1,nb.config.NUMBA_NUM_THREADS+1) if x in [1,4,8] or x%8 == 0]
compilers = ['vs','intel'] if DO_INTEL else ['vs']

# Calling C++

In [4]:
model = BufferStockModelClass(name='',par={'solmethod':'egm'})

**Extra requirement:** `.settings()` should define `.cpp_filename` and `.cpp_options`.

1. All elements in `namespaces` are availible as C++ structs with `_struct` appended (default, else specify `.cpp_structsmap` in `.settings()`).
2. The C++ file is linked with `.link_to_cpp()`, and is then available in `.cpp`
3. `.cpp.delink()`: Delink C++ file (release the .dll file, so that it can be overwritten when e.g. re-compiling).
4. `.cpp.recompile()`: Delink, re-compile and re-link to C++ file.
5. `.cpp.clean_up()`: Delink, remove .dll file, and remove any struct files.

In [5]:
for compiler in compilers:
    
    # a. compile
    model.cpp_options['compiler'] = compiler # change compiler
    model.link_to_cpp(do_print=(compiler=='vs'))
    
    print(f'### compiler = {compiler} ###')
    print('')
        
    # b. run with different number of threads
    for threads in threads_list:
        
        model.par.cppthreads = threads
        tic,toc = model.solve_cpp()
        
        print(f' time to solve with {threads:2d} threads {toc-tic:.2f} secs [checksum: {model.checksum():.8f}]')    
    
    print('')
        
    # c. unlink
    model.cpp.delink() # else it is not possible to compile again

Linking to: cppfuncs/egm.cpp

### finding all included files ###

logs.cpp
par_struct.cpp
sol_struct.cpp
sim_struct.cpp

### writing structs ###

cppfuncs/sol_struct.cpp
 double* c;
 double* v;
 double* w;
 double* q;

cppfuncs/par_struct.cpp
 char* solmethod;
 int T;
 double beta;
 double rho;
 double R;
 double sigma_psi;
 int Npsi;
 double sigma_xi;
 int Nxi;
 double pi;
 double mu;
 int Nm;
 int Np;
 int Na;
 double tol;
 bool do_print;
 bool do_simple_w;
 int cppthreads;
 int simT;
 int simN;
 int sim_seed;
 double* grid_m;
 double* grid_p;
 double* grid_a;
 double* psi;
 double* psi_w;
 double* xi;
 double* xi_w;
 int Nshocks;

cppfuncs/sim_struct.cpp
 double* p;
 double* m;
 double* c;
 double* a;
 double* psi;
 double* xi;

### analyzing cppfuncs/egm.cpp ###

function: solve
return type: void
argument types: ['par_struct*', 'sol_struct*']

### analyzing cppfuncs/logs.cpp ###

### analyzing cppfuncs/par_struct.cpp ###

### analyzing cppfuncs/sol_struct.cpp ###

### analyzing cpp

# Clean-up

In [6]:
model.cpp.clean_up()